In [146]:
import os
import sys
from glob import glob

In [193]:
import shutil
import zipfile

In [235]:
class MoveFile:
    """想定するファイル構成

    <DataSet RootDir>
        |
        |-- <original>

    """
    def __init__(self, parent_path):
        # 操作したいデータセットのrootディレクトリ
        self.parent_path = parent_path
        # 
        self.org_path = os.path.join(self.parent_path, 'original')

    def unpack(self, file_name: str, create_dir: bool=True):
        """originalディレクトリ内のzipファイルを展開するための関数
        Arg:
            file_name(str): original ディレクトリ内の zip ファイル名,
            create_dir(bool optional): 解凍するときに、解凍前のファイル名と同じ名前のディレクトリを作成する。default to True.
        """
        zip_path = os.path.join(self.org_path, file_name + '.zip')
        unpack_path = self.parent_path

        if create_dir:
            try:
                unpack_path = os.path.join(self.parent_path, file_name)
                os.mkdir(unpack_path) # create directry
            except FileExistsError as e:
                print("ディレクトリ作成時にエラーが発生しました: ", e)
                return
            
        with zipfile.ZipFile(zip_path) as existing_zip:
            existing_zip.extractall(unpack_path)
    
    def MoveImage(self, src: str, dst: str):
        """子ディレクトリ内に存在するファイルを別の子ディレクトリ内の正数で表されたディレクトリ内に移動する関数
        Args:
            src(str): 移動させたいファイルやディレクトリのパス,
            dst(str): 移動先のディレクトリのパス

        <Root Dir>
        |-- 0a7d3O
        |   |-- images
        |   |     |-- 0a7d3O.png
        |   |
        |   |-- masks
        |         |-- 0adbf5.png
        |         |-- 1ed5b9.png
        |         |-- 3bef1O.png
        |         |-- ...
        |
        |-- 0acd2c
        |   |-- images
        |   |     |-- 0acd2c.png
        |   |
        |   |-- masks
        |         |-- 2f5eb1.png
        |         |-- 3fa2a8.png
        |         |-- O5fefb.png
        |         |-- ...
        |
        |-- 00ae65c
        |   |-- ...
        |
        |-- ...
        """
        dst_img_path = os.path.join(dst, 'images')
        dst_mask_path = os.path.join(dst, 'masks')
        try:
            os.mkdir(dst_img_path)  # images ディレクトリを作る
            os.mkdir(dst_mask_path) #  masks ディレクトリを作る
        except FileExistsError as e:
            print("ディレクトリ作成時にエラーが発生しました: ", e)
            pass
        #list_file_and_dir_name = os.listdir(curent_path)
        
        #list_file_and_dir_path = glob(os.path.join(src, '**', '*'))
        #list_file_and_dir_path = glob(os.path.join(src, '**'), recursive=True)

        # 全ての子ディレクトリの相対パスを取得 <Root> -> <0a7d330>
        list_dir_path = glob(os.path.join(src, '**'))
        for dir_path in list_dir_path:
            # 子ディレクトリ内の images ディレクトリの相対パスを取得 <0a7d330> -> images
            list_img_dir_path = [f for f in glob(os.path.join(dir_path, '**')) if 'images' in f]
            for img_dir_path in list_img_dir_path:
                list_file = os.listdir(img_dir_path)
                if list_file: # ディレクトリ内にファイルが存在するか判定
                    for p in list_file:
                        # ディレクトリ内の全てのファイルを移動
                        shutil.move(os.path.join(img_dir_path, p), dst_img_path)

            # 子ディレクトリ内の masks ディレクトリの相対パスを取得 <0a7d330> -> masks
            list_masks_dir_path = [f for f in glob(os.path.join(dir_path, '**')) if 'masks' in f]
            for masks_dir_path in list_masks_dir_path:
                list_file = os.listdir(masks_dir_path)
                if list_file: # ディレクトリ内にファイルが存在するか判定
                    for num, p in enumerate(list_file):
                        dst_path = os.path.join(dst_mask_path, str(num))
                        if not os.path.exists(dst_path):
                            os.mkdir(dst_path)
                            shutil.move(os.path.join(masks_dir_path, p), dst_path)
                        else:
                            shutil.move(os.path.join(masks_dir_path, p), dst_path)
                            


In [236]:
sys.path.append(".")
sys.path.append("..")
sys.path.append("../../")
from src.config.config import cfg

In [237]:
m_file = MoveFile(cfg.DATA_SCIENCE_BOWL_DIR)

In [238]:
# f_name = 'stage1_sample_submission.csv' # 解凍したいファイル名
f_name = 'stage1_train'
m_file.unpack(f_name)

ディレクトリ作成時にエラーが発生しました:  [WinError 183] 既に存在するファイルを作成することはできません。: 'd:\\My_programing\\python\\AI\\Analysis\\data\\DataScienceBowl\\stage1_train'


In [239]:
src = os.path.join(cfg.DATA_SCIENCE_BOWL_DIR, 'stage1_train')
dst = cfg.DSB2018_96_DIR
m_file.MoveImage(src, dst)

ディレクトリ作成時にエラーが発生しました:  [WinError 183] 既に存在するファイルを作成することはできません。: 'd:\\My_programing\\python\\AI\\Analysis\\data\\DataScienceBowl\\dsb2018_96\\images'
